In [ ]:
import torch
import torchvision.transforms as transforms
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import matplotlib.pyplot as plt
import numpy as np
import glob as glob
from torch import topk
import pandas as pd
from collections.abc import Iterable
from torch.utils.data import Dataset, DataLoader
import itertools
import tqdm
import copy
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.metrics import mean_squared_error
from sklearn import preprocessing
import seaborn as sns
# imports from captum library
# from captum.attr import LayerConductance, LayerActivation, LayerIntegratedGradients
# from captum.attr import IntegratedGradients, DeepLift, GradientShap, NoiseTunnel, FeatureAblation
from sklearn.metrics import f1_score
from torch.utils.tensorboard import SummaryWriter
from collections import Counter
import datetime

In [ ]:
def apply_func_adapted(row,dist_max):
    if '..' in row['picture_name']:
        for i in range(0, 8):
            if i % 2 == 0:
                row[i] = row[i] / 640
            else:
                row[i] = row[i] / 480
        
        for i in range(8, 71):
            if i % 3 == 2:
                row[i] = row[i] / 640
            elif i % 3 == 0:
                row[i] = row[i] / 480
        row[71] = row[71] / dist_max
    
    else:
        for i in range(0, 8):
            if i % 2 == 0:
                row[i] = row[i] / 1920
            else:
                row[i] = row[i] / 1080
    
        for i in range(8, 71):
            if i % 3 == 2:
                row[i] = row[i] / 1920
            elif i % 3 == 0:
                row[i] = row[i] / 1080
        row[71] = row[71] / dist_max
    
    return row

In [ ]:
class Data(Dataset):
    def __init__(self, X, y):
        self.X = torch.from_numpy(X.astype(np.float32))
        self.y = torch.from_numpy(y.astype(np.float32))
        self.len = self.X.shape[0]
       
    def __getitem__(self, index):
        return self.X[index], self.y[index]
   
    def __len__(self):
        return self.len
    
class Net(nn.Module):
    def __init__(self, input_dim, hidden_dim_1, hidden_dim_2, output_dim):
        super(Net, self).__init__()
        self.layer_1 = nn.Linear(input_dim, hidden_dim_1)
        self.layer_2 = nn.Linear(hidden_dim_1, hidden_dim_2)
        self.layer_3 = nn.Linear(hidden_dim_2,output_dim)
       
        self.dropout = nn.Dropout(0.2)

    def forward(self, x):
        x = torch.nn.functional.relu(self.layer_1(x))
        x = self.dropout(x)
        x = torch.nn.functional.relu(self.layer_2(x))
        x = self.layer_3(x)

        return x

In [ ]:
input_dim = 74
hidden_dim_1 = 64
hidden_dim_2 = 32
output_dim = 3

model = Net(input_dim, hidden_dim_1, hidden_dim_2, output_dim)

checkpoint = torch.load('modelval6.pth')

model.load_state_dict(checkpoint['model_state_dict'])

In [ ]:
df = pd.read_csv('end_val.csv')

In [ ]:
X = df.iloc[:, 0:-1]
y = df.iloc[:, -1:]

X = pd.get_dummies(X, columns=['handedness'])
ohe = OneHotEncoder(handle_unknown='ignore', sparse_output=False).fit(y)
y = ohe.transform(y)


In [ ]:
max_dist_train = X['depth_dist'].abs().max()

X = X.apply(apply_func_adapted, axis=1, args=([max_dist_train]))

X.drop('picture_name', axis=1, inplace=True) 


X = torch.tensor(X.values, dtype=torch.float32)

In [ ]:
test_data = Data(np.array(X), np.array(y))
test_dataloader = DataLoader(dataset=test_data, batch_size=32, shuffle=False)

In [ ]:
y_pred = []
y_true = []
y_name = []

for inputs, labels in test_dataloader:
    output = model(inputs) # Feed Network

    output = (torch.max(torch.exp(output), 1)[1]).data.cpu().numpy()
    y_pred.extend(output) # Save Prediction
    
    labels = labels.data.cpu().numpy()
    y_true.extend(labels) # Save Truth

y_true = [np.argmax(i) for i in y_true]


In [ ]:
element_count = Counter(y_true)
print('0:none', '1:power', '2:precision')
print(element_count)
print()
print('F1', f1_score(y_true, y_pred, average=None))
print('F1-micro', f1_score(y_true, y_pred, average='micro'))
print('F1-macro', f1_score(y_true, y_pred, average='macro'))
print('F1-weighted', f1_score(y_true, y_pred, average='weighted'))

In [ ]:
classes = ('none', 'power', 'precision')

cf_matrix = confusion_matrix(y_true, y_pred)
df_cm_perc = pd.DataFrame(cf_matrix / np.sum(cf_matrix, axis=1), index = [i for i in classes],
                     columns = [i for i in classes])
df_cm_int = pd.DataFrame(cf_matrix , index = [i for i in classes],
                     columns = [i for i in classes])
plt.figure(figsize = (12,7))
sns.heatmap(df_cm_perc, annot=True, cmap='Blues',annot_kws={"fontsize": 20})
plt.savefig('output_perc_val1.png')
plt.figure(figsize = (12,7))
sns.heatmap(df_cm_int, annot=True, cmap='Blues',fmt="d",annot_kws={"fontsize": 20})
plt.savefig('output_int_val1.png')